# Zero Shot Text Classification Model
**Model Description** - Bart with a classification head trained on MNLI.

Sequences are posed as NLI premises and topic labels are turned into premises, i.e. business -> This text is about business.

In [1]:
import eland as ed
import pandas as pd
import numpy as np
import preprocessor as prep

from sklearn.ensemble import RandomForestClassifier
from src.models import ALZeroShotWrapper

ed_df = ed.read_es('localhost', 'twitter')

2020-08-26 16:16:01,188 - elasticsearch - INFO - GET http://localhost:9200/twitter/_mapping [status:200 request:0.033s]
2020-08-26 16:16:01,207 - elasticsearch - INFO - GET http://localhost:9200/twitter/_field_caps?fields=%2A [status:200 request:0.017s]


## Importing Data

In [2]:
df = ed_df[
    ['tweet_id', 'lang', 'full_text']
].to_pandas().fillna(np.nan)

2020-08-26 16:16:02,048 - elasticsearch - INFO - POST http://localhost:9200/twitter/_search?scroll=5m&size=1000 [status:200 request:0.547s]
2020-08-26 16:16:02,484 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.417s]
2020-08-26 16:16:03,032 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.518s]
2020-08-26 16:16:03,271 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.228s]
2020-08-26 16:16:03,485 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.203s]
2020-08-26 16:16:03,650 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.153s]
2020-08-26 16:16:03,820 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.158s]
2020-08-26 16:16:04,002 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.170s]
2020-08-26 16:16:04,146 - e

2020-08-26 16:16:16,083 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.202s]
2020-08-26 16:16:16,233 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.136s]
2020-08-26 16:16:16,393 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.143s]
2020-08-26 16:16:16,527 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.119s]
2020-08-26 16:16:16,711 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.165s]
2020-08-26 16:16:16,823 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.095s]
2020-08-26 16:16:17,048 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.212s]
2020-08-26 16:16:17,418 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.350s]
2020-08-26 16:16:17,541 - elasticsearch - INFO -

2020-08-26 16:16:27,870 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.141s]
2020-08-26 16:16:28,029 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.146s]
2020-08-26 16:16:28,149 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.106s]
2020-08-26 16:16:28,290 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.126s]
2020-08-26 16:16:28,421 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.112s]
2020-08-26 16:16:28,545 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.113s]
2020-08-26 16:16:28,688 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.130s]
2020-08-26 16:16:28,812 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.107s]
2020-08-26 16:16:28,941 - elasticsearch - INFO -

2020-08-26 16:16:39,460 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.150s]
2020-08-26 16:16:39,694 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.212s]
2020-08-26 16:16:39,859 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.135s]
2020-08-26 16:16:39,994 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.121s]
2020-08-26 16:16:40,120 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.112s]
2020-08-26 16:16:40,272 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.132s]
2020-08-26 16:16:40,416 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.131s]
2020-08-26 16:16:40,530 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.102s]
2020-08-26 16:16:40,679 - elasticsearch - INFO -

2020-08-26 16:16:50,874 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.136s]
2020-08-26 16:16:50,991 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.104s]
2020-08-26 16:16:51,113 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.107s]
2020-08-26 16:16:51,218 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.089s]
2020-08-26 16:16:51,331 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.099s]
2020-08-26 16:16:51,493 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.146s]
2020-08-26 16:16:51,631 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.119s]
2020-08-26 16:16:51,772 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.128s]
2020-08-26 16:16:54,658 - elasticsearch - INFO -

2020-08-26 16:17:10,893 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.152s]
2020-08-26 16:17:11,218 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.106s]
2020-08-26 16:17:11,377 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.138s]
2020-08-26 16:17:11,525 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.134s]
2020-08-26 16:17:11,690 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.148s]
2020-08-26 16:17:11,901 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.187s]
2020-08-26 16:17:12,119 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.198s]
2020-08-26 16:17:12,262 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.122s]
2020-08-26 16:17:12,394 - elasticsearch - INFO -

2020-08-26 16:17:23,180 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.163s]
2020-08-26 16:17:23,331 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.124s]
2020-08-26 16:17:23,516 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.168s]
2020-08-26 16:17:23,693 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.156s]
2020-08-26 16:17:23,918 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.195s]
2020-08-26 16:17:24,109 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.157s]
2020-08-26 16:17:24,455 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.140s]
2020-08-26 16:17:24,812 - elasticsearch - INFO - POST http://localhost:9200/_search/scroll [status:200 request:0.316s]
2020-08-26 16:17:25,057 - elasticsearch - INFO -

In [3]:
tweet_vectors = pd.read_csv('../data/results/iwmi_tweet2vec.csv')

tweet_vectors['tweet_id'] = tweet_vectors['tweet_id'].astype(float).astype(int).astype(str)
df['tweet_id'] = df['tweet_id'].astype(float).astype(int).astype(str)

df = df.set_index('tweet_id').join(tweet_vectors.set_index('tweet_id'))

In [4]:
df

,lang,full_text,original_tweet_id_str,user_id,name,full_text_processed,sentiment,followers_count,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9
tweet_id,,,,,,,,,,,,,,,,,,
1263737818767130624,hi,@ExSecular DD ki gaan uda di Amphan ne. Ab bhi...,NaN,830636185869545472,Nikki 🇮🇳,exsecular dd ki gaan uda di amphan ne ab bhikh...,0.0000,20,-0.092787,-0.494704,-0.076695,-0.306650,0.175336,-0.084080,-1.023983,-0.352764,0.190316,0.652073
1263737812576202752,en,Speaking on the situation in the wake of Cyclo...,1.263735e+18,1121047353899622400,A $ H W I K K 🇮🇳,speaking situation wake cyclone amphan,0.0000,771,-0.228025,-0.639948,0.130544,-0.116342,0.107413,0.055122,-0.713540,-0.028307,0.130019,0.491384
1263737811460636672,en,Speaking on the situation in the wake of Cyclo...,1.263735e+18,1028668637865947136,Vandana. pooja,speaking situation wake cyclone amphan,0.0000,27950,-0.253947,-0.727148,0.058120,-0.067041,0.177521,0.123356,-0.685045,-0.126759,0.098336,0.481952
1263737810105839616,en,Speaking on the situation in the wake of Cyclo...,1.263735e+18,896275238182244352,মৃদুল ফুকন 🇮🇳 मृदुल फुकन 🇮🇳 Mridul Phookan,speaking situation wake cyclone amphan,0.0000,209,-0.090962,-0.734353,0.054442,-0.065879,0.160751,0.118595,-0.788140,-0.095145,0.157350,0.509983
1263737809757749248,en,The damage to the Sunderbans by #AmphanCyclone...,1.263694e+18,4271835029,MD YOUNUS KHAN,damage sunderbans amphancyclone last quite lon...,-0.6808,7,0.121170,-0.579570,-0.351209,0.132984,0.328902,0.050509,-1.068896,1.050037,-0.302412,0.448421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262711874832994304,en,Cyclone Amphan has yet to make landfall but is...,NaN,1877831,New York Times World,cyclone amphan ha yet make landfall already la...,0.6908,2061963,-0.190201,0.132993,-0.059034,-0.054390,0.195255,-0.303061,-1.309998,-0.036518,-0.251265,0.281315
1254077710600679424,en,UN: 'Our fear is tremendous loss of life…women...,NaN,254206787,Nicola Careem,un fear tremendous loss life…women children…in...,-0.9062,16468,0.745186,0.723378,-0.726585,0.051456,-0.058524,0.619988,-0.231784,0.468151,-0.621294,1.725290
1262672672946900992,en,#MANITSaysNoExams\n#PromoteFinalYearStudents\n...,NaN,1262411428994015232,NIT Bhopal Voice,manitsaysnoexams promotefinalyearstudents mhrd...,0.7184,631,0.258051,-0.727731,-0.139681,-0.024986,0.274035,0.584165,0.881894,-0.024248,-0.791794,0.324785


## Basic Tweet Preprocessing
- Remove URLs and reserved words (RTs)
- Remove # and @ symbols
- Remove tweets less than 4 tokens in length


In [5]:
## Set options for the tweet-preprocessor
prep.set_options(prep.OPT.URL, prep.OPT.RESERVED, prep.OPT.EMOJI, prep.OPT.SMILEY)

## Clean text and remove #,@ symbols
def clean_tweet(text):
    text = prep.clean(text)
    table = str.maketrans('','','#@')
    return text.translate(table)

In [6]:
df['full_text'] = df['full_text'].apply(lambda x: clean_tweet(x))

In [7]:
df['length'] = df['full_text'].apply(lambda x: len(x.split()))
df = df[df['length']>4]

In [8]:
df_train = df[df.lang == 'en']
X = df_train[[f'vec_{i}' for i in range(10)]].values
sequences = df_train.full_text.values
candidate_labels = [
    'resource availability', 
    'volunteers', 
    'power supply', 
    'relief measures', 
    'food supply', 
    'infrastructure', 
    'medical assistance', 
    'rescue', 
    'shelter', 
    'utilities', 
    'water supply', 
    'evacuation', 
    'government', 
    'crime violence', 
    'mobile network', 
    'sympathy', 
    'news updates', 
    'internet', 
    'grievance'
]


## Initialising the Model

In [11]:
rfc = RandomForestClassifier(random_state=0)
al_zeroshot = ALZeroShotWrapper(
    rfc, 
    selection_strategy='random', 
    max_iter=300, 
    n_initial=20, 
    increment=20, 
    random_state=0,
    verbose=True
)

al_zeroshot.fit(X, sequences, candidate_labels)

2020-08-26 17:21:10,847 - src.models._al_zscf - INFO - Started fitting process. Setting up models.
2020-08-26 17:21:11,632 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-mnli/config.json from cache at /Users/jfonseca/.cache/torch/transformers/a35b79dc26c2f371a0e19eae44d91c0a0281a5db09044517d2675703791ee3c5.746d7ef19ade685cd3ee03f131a96fab513947c26179546289ddf02a6ac683ce
2020-08-26 17:21:11,640 - transformers.configuration_utils - INFO - Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder

2020-08-26 18:52:21,427 - src.models._al_zscf - INFO - Iteration #38
2020-08-26 18:54:40,638 - src.models._al_zscf - INFO - Iteration #39
2020-08-26 18:56:49,150 - src.models._al_zscf - INFO - Iteration #40
2020-08-26 18:59:20,363 - src.models._al_zscf - INFO - Iteration #41
2020-08-26 19:01:41,312 - src.models._al_zscf - INFO - Iteration #42
2020-08-26 19:04:19,999 - src.models._al_zscf - INFO - Iteration #43
2020-08-26 19:06:49,968 - src.models._al_zscf - INFO - Iteration #44
2020-08-26 19:09:00,266 - src.models._al_zscf - INFO - Iteration #45
2020-08-26 19:11:05,671 - src.models._al_zscf - INFO - Iteration #46
2020-08-26 19:13:33,397 - src.models._al_zscf - INFO - Iteration #47
2020-08-26 19:15:46,572 - src.models._al_zscf - INFO - Iteration #48
2020-08-26 19:18:02,527 - src.models._al_zscf - INFO - Iteration #49
2020-08-26 19:20:23,607 - src.models._al_zscf - INFO - Iteration #50
2020-08-26 19:22:48,356 - src.models._al_zscf - INFO - Iteration #51
2020-08-26 19:25:19,463 - src.mode

2020-08-26 23:34:11,264 - src.models._al_zscf - INFO - Iteration #156
2020-08-26 23:36:19,359 - src.models._al_zscf - INFO - Iteration #157
2020-08-26 23:39:08,447 - src.models._al_zscf - INFO - Iteration #158
2020-08-26 23:41:06,004 - src.models._al_zscf - INFO - Iteration #159
2020-08-26 23:43:24,782 - src.models._al_zscf - INFO - Iteration #160
2020-08-26 23:45:50,139 - src.models._al_zscf - INFO - Iteration #161
2020-08-26 23:48:07,544 - src.models._al_zscf - INFO - Iteration #162
2020-08-26 23:50:37,947 - src.models._al_zscf - INFO - Iteration #163
2020-08-26 23:52:39,939 - src.models._al_zscf - INFO - Iteration #164
2020-08-26 23:54:50,252 - src.models._al_zscf - INFO - Iteration #165
2020-08-26 23:57:16,207 - src.models._al_zscf - INFO - Iteration #166
2020-08-26 23:59:51,651 - src.models._al_zscf - INFO - Iteration #167
2020-08-27 00:02:10,309 - src.models._al_zscf - INFO - Iteration #168
2020-08-27 00:04:30,742 - src.models._al_zscf - INFO - Iteration #169
2020-08-27 00:06:45,

2020-08-27 12:05:30,387 - src.models._al_zscf - INFO - Iteration #274
2020-08-27 12:08:51,987 - src.models._al_zscf - INFO - Iteration #275
2020-08-27 12:12:32,030 - src.models._al_zscf - INFO - Iteration #276
2020-08-27 12:16:00,262 - src.models._al_zscf - INFO - Iteration #277
2020-08-27 12:19:11,420 - src.models._al_zscf - INFO - Iteration #278
2020-08-27 12:22:53,583 - src.models._al_zscf - INFO - Iteration #279
2020-08-27 12:26:09,913 - src.models._al_zscf - INFO - Iteration #280
2020-08-27 12:29:54,343 - src.models._al_zscf - INFO - Iteration #281
2020-08-27 12:33:19,629 - src.models._al_zscf - INFO - Iteration #282
2020-08-27 12:36:40,341 - src.models._al_zscf - INFO - Iteration #283
2020-08-27 12:39:43,464 - src.models._al_zscf - INFO - Iteration #284
2020-08-27 12:42:40,008 - src.models._al_zscf - INFO - Iteration #285
2020-08-27 12:46:06,252 - src.models._al_zscf - INFO - Iteration #286
2020-08-27 12:49:17,851 - src.models._al_zscf - INFO - Iteration #287
2020-08-27 12:52:45,

ALZeroShotWrapper(classifier=RandomForestClassifier(random_state=0),
                  increment=20, max_iter=300, n_initial=20, random_state=0,
                  selection_strategy='random')

In [26]:
zero_shot_labels = al_zeroshot.y

rfc_labels = al_zeroshot\
    .label_encoder\
    .inverse_transform(
        al_zeroshot\
            .classifier_\
            .predict(X)\
            .astype(int)
    )

In [31]:
zs_mask = ~np.isnan(zero_shot_labels)
ground_truth_labels = al_zeroshot\
    .label_encoder\
    .inverse_transform(
        zero_shot_labels[zs_mask]\
            .astype(int)
    )
rfc_zs_preds = rfc_labels[zs_mask]

In [34]:
# Checking if the random forest was well trained on pseudo-ground-truth data
(ground_truth_labels == rfc_zs_preds).all()

True

## Classify the entire dataset using the trained random forest

In [35]:
le = al_zeroshot.label_encoder
rfc = al_zeroshot.classifier_

In [43]:
X_all = df[[f'vec_{i}' for i in range(10)]].values
y_pred = le.inverse_transform(rfc.predict(X_all).astype(int))

In [92]:
df['label_prediction'] = y_pred
df = df.join(
    pd.Series(
        zero_shot_labels.astype(int), 
        name='zero_shot_labels', 
        index=df_train.reset_index()['tweet_id']
    )
)

df.loc[df.zero_shot_labels < 0, 'zero_shot_labels'] = np.nan

df.loc[~df.zero_shot_labels.isnull(), 'zero_shot_labels'] = le.inverse_transform(
        df[~df.zero_shot_labels.isnull()]['zero_shot_labels'].astype(int)
)

df[['full_text', 'label_prediction']]

,full_text,label_prediction
tweet_id,,
1263737818767130624,ExSecular DD ki gaan uda di Amphan ne. Ab bhik...,grievance
1263737812576202752,Speaking on the situation in the wake of Cyclo...,relief measures
1263737811460636672,Speaking on the situation in the wake of Cyclo...,relief measures
1263737810105839616,Speaking on the situation in the wake of Cyclo...,relief measures
1263737809757749248,The damage to the Sunderbans by AmphanCyclone ...,grievance
...,...,...
1262711874832994304,Cyclone Amphan has yet to make landfall but is...,grievance
1254077710600679424,UN: 'Our fear is tremendous loss of lifewomen ...,evacuation
1262672672946900992,MANITSaysNoExams PromoteFinalYearStudents MHRD...,relief measures


In [46]:
df.to_csv('../data/results/provisional_topic_modelling.csv')

In [ ]:
i = 14
print(df.iloc[i].full_text)
print(df.iloc[i].label_prediction)

In [102]:
#df[~df.zero_shot_labels.isnull()]['zero_shot_labels'].shape

